In [ ]:
#Bow  Bag of Words  CountVectorizer
# 문서를 고정된 길이의 벡터로 변환
# 문서 - 단어행렬
# 장점 : 간단하고 빠름,
# 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

# tf-idf TfidfVectorizer
# 모든 문서에서 자주등장하는 단어의 영향을 줄이고 문서 특이 단어를 강조

# multinomal Naive Bayes  확률 모델
# LogisticRegression : 다중클래스  회귀기반 분류

# RidgeClassifer : 회귀 기반 분류 L2규제

# N-gram : 단점 차원폭발에 주의 (정규화/차원 축소 고려)

# Kolnpy Okt


In [ ]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# # data load
newsgroups_train =  fetch_20newsgroups(subset='train'
                    ,remove = ('headers','footers','quotes')
                    ,categories=categories
                   )
newsgroups_test =  fetch_20newsgroups(subset='test'
                    ,remove = ('headers','footers','quotes')
                    ,categories=categories
                   )

categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

- alt.atheism	무신론, 종교 비판 토론	종교의 존재, 신의 존재 유무, 종교적 주장 반박, 철학적 논쟁 등
- talk.religion.misc	일반 종교 토론 (기타 잡담 포함)	기독교, 불교, 이슬람 등 다양한 종교 관련 이야기, 개인 경험, 신념 공유 등
- comp.graphics	컴퓨터 그래픽스, 이미지 처리	3D 렌더링, 이미지 파일 포맷, 그래픽 소프트웨어 사용법, OpenGL 등 기술 관련 토론
- sci.space 우주 과학, 천문학 로켓, NASA, 행성 탐사, 외계 생명 가능성, 우주 물리학 관련 토론

In [ ]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [ ]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

In [ ]:
# 해더 푸터 인용문 제거

In [ ]:
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text

In [ ]:
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [ ]:
len(train_data), len(train_target), len(test_data), len(test_target)

- 멀티노멀 나이즈베이즈
- 문서에 포함된 단어들의 출현 횟수를 기반으로 해서 그 문서가 어떤 주제에 속할지 확률적
- 스팸필터링, 뉴스기사 카테고리, 감성분석

- 베이즈정리 확률 이론 - 조건부 확률
- 단어A가 나왔을때 이 문서가 스팸 B 일 확률은 얼마

$P(\text{스팸} | \text{단어들}) = \frac{P(\text{단어들} | \text{스팸}) \cdot P(\text{스팸})}{P(\text{단어들})}$

- 나이브 Naive  : 순진한 가정
    - 가정 : 문서안의 모든 단어는 서로 독립적
    - 현실 : 스펨에 자주 나오는 단어들은 서로 독립적이지 않다
    - 실제 : 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느정도 나온다
- 멀티노멀 : 다항 분포
    - 의미 : 단어의 출현 횟수를 중요
    - 횟수를 세는 멀티노미얼 방식이 NLP 잘 맞는다
    - 모델은 단어의 빈도수 통계
    - 스펨메일 통계 spem
        - free : 150
        - money : 100
        - viagra : 50
        - report : 5
    - 정상메일 Ham 통계
        - report:80
        - metting : 60
        - free : 10
    - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 P('free'|스펨) 을 모두 계산
    - "Free money meeting"
        - 스펨??
            - 기본스펨확률 x 스팸일때 free가 나올 확률 x 스팸일때 money 나올 확률 x 스팸일때 meeting 나올 확률
        - 정상
            - 기본정상확률 x 정상일때...   

In [ ]:
# nltk tokenizer stemer
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

In [ ]:
# min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
# max_df : 50% 너무 흔한 단어는 제거
cv = CountVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.fit_transform(test_data)

In [ ]:
cv.get_feature_names_out()

In [ ]:
x_train_cv[0].toarray()[0]

In [ ]:
# BOW 기반
# 텍스트 분류의 강력한 baseline 희소데이터에 강함
# 모델선택
nb = MultinomialNB()
# 학습용 데이터, 벡터 데이터
nb.fit(x_train_cv, train_target)
print(nb.score(x_train_cv, train_target), nb.score(x_test_cv, test_target))
# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print( classification_report(test_target, y_pred_nb, target_names=categories) )

In [ ]:
# TF-IDF + MNB + LogisticRegression
# TF-IDF로 중요단어 강조, 선형모델과 자주 사용  BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfid = tfidf.fit_transform(train_data)
x_test_tfid = tfidf.transform(test_data)

# NB + tf-idf
nb_tfidf = MultinomialNB()
nb_tfidf.fit(x_train_tfid, train_target)
print(nb_tfidf.score(x_train_tfid, train_target), nb_tfidf.score(x_test_tfid, test_target))

In [ ]:
# logistic
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train_tfid, train_target, test_size=0.2, stratify=train_target, random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train, y_train), lr.score(x_test, y_test))

In [ ]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=10)
rc.fit(x_train, y_train)
print(rc.score(x_train, y_train), rc.score(x_test, y_test))

In [ ]:
# L1 규제   L1 Logistic(Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행..
l1_lr = LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear')
l1_lr.fit(x_train, y_train)
print(l1_lr.score(x_train, y_train), l1_lr.score(x_test, y_test))

In [ ]:
# 트리모델 + tfidf
tree = DecisionTreeClassifier()
forest = RandomForestClassifier()
gb = GradientBoostingClassifier()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target, stratify=train_target, test_size=0.2, random_state=42)

In [ ]:
# 전처리
# RegexpTokenizer + stopwords + PorterStemmer
english_stops = set(stopwords.words('english'))
regtok = RegexpTokenizer(r"[\w]{3,}")
regtok.tokenize('i love you')
def custom_tokenizer(text):
    toks = regtok.tokenize(text.lower())
    toks = [t for t in toks if t not in english_stops]
    tokes = [PorterStemmer().stem(t) for t in toks]
    return tokes
tfidf_custom = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf_c = tfidf_custom.fit_transform(x_train)
x_test_tfidf_c = tfidf_custom.transform(x_test)

In [ ]:
lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_tfidf_c, y_train)
print( lr_c.score(x_train_tfidf_c, y_train), lr_c.score(x_test_tfidf_c, y_test))

In [ ]:
# n-gram 실험 1,2  1,3
# 성능향상 기대  연속된 단어패턴 포착
tfidf_12 = TfidfVectorizer(token_pattern = r"[\w]){3,}", 
                           stop_words=stopwords.words('english'),
                           ngram_range=(1,2),
                           min_df=2, max_df=0.5,
                           max_features=2000
                           )
x_train_12 = tfidf_12.fit_transform(x_train)
x_test_12 = tfidf_12.transform(x_test)

lr_c = LogisticRegression(max_iter=1000)
lr_c.fit(x_train_12, y_train)
print( lr_c.score(x_train_12, y_train), lr_c.score(x_test_12, y_test))

In [108]:
# 한국어 처리  konlpy
# 품사기반 태깅 tokenizer   Noun, Verb, Adjective
# 데이터 로딩
import pandas as pd
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [110]:
df.title.unique()

array(['인피니티 워', '라라랜드', '곤지암', '신과함께', '범죄도시', '택시운전사', '코코'],
      dtype=object)

In [113]:
x_train, x_test, y_train, y_test = train_test_split(df.review, df.title, stratify=df.title, test_size=0.2, random_state=42)

In [116]:
from konlpy.tag import Okt 
okt = Okt()

In [118]:
# simple version
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_tfidf, y_train)
print(clf.score(x_train_tfidf, y_train), clf.score(x_test_tfidf, y_test))

c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.7565365025466892 0.6899830220713074


In [120]:
# simple version
okt = Okt()
def custom_tokenizer(text):
    target = ['Noun', 'Verb', 'Adjective']
    return [w for w,tag in okt.pos(text, norm=True, stem=True) if tag in target]

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=2000, min_df=5, max_df=0.5)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)
clf = LogisticRegression(max_iter=1000)
clf.fit(x_train_tfidf, y_train)
print(clf.score(x_train_tfidf, y_train), clf.score(x_test_tfidf, y_test))

c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.782258064516129 0.7137521222410866
